In [4]:


import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Titanic 데이터셋을 위한 사용자 정의 Dataset 클래스
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    # 입력 데이터와 타겟 데이터를 PyTorch Tensor로 변환
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    # 데이터셋의 크기를 반환
    return len(self.X)

  def __getitem__(self, idx):
    # 주어진 인덱스의 데이터를 반환
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    # 데이터셋의 크기와 모양을 문자열로 반환
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

# 테스트 데이터셋을 위한 사용자 정의 Dataset 클래스
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    # 입력 데이터만 PyTorch Tensor로 변환
    self.X = torch.FloatTensor(X)

  def __len__(self):
    # 데이터셋의 크기를 반환
    return len(self.X)

  def __getitem__(self, idx):
    # 주어진 인덱스의 데이터를 반환
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    # 데이터셋의 크기와 모양을 문자열로 반환
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

# 데이터 전처리를 수행하여 훈련, 검증, 테스트 데이터셋을 반환하는 함수
def get_preprocessed_dataset():
    # 현재 작업 디렉토리 경로를 가져옴
    CURRENT_FILE_PATH = os.getcwd()

    # 학습 및 테스트 데이터 파일 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # CSV 파일을 읽어와 데이터프레임으로 변환
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # 학습과 테스트 데이터를 결합하여 전처리 수행
    all_df = pd.concat([train_df, test_df], sort=False)

    # 각 전처리 단계 함수 호출
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # 학습 데이터와 타겟을 분리
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    # 테스트 데이터 분리
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # PyTorch Dataset 생성
    dataset = TitanicDataset(train_X.values, train_y.values)
    # 훈련 데이터셋과 검증 데이터셋을 8:2 비율로 나눔
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    # 테스트 데이터셋 생성
    test_dataset = TitanicTestDataset(test_X.values)

    return train_dataset, validation_dataset, test_dataset

# 전처리 단계 1: Pclass별로 Fare의 결측치를 평균값으로 채움
def get_preprocessed_dataset_1(all_df):
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

# 전처리 단계 2: 이름을 성, 호칭, 이름으로 분리
def get_preprocessed_dataset_2(all_df):
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

# 전처리 단계 3: 호칭별 나이의 결측치를 중앙값으로 채움
def get_preprocessed_dataset_3(all_df):
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean"]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

# 전처리 단계 4: 가족 수와 혼자 탑승 여부 컬럼 추가 및 불필요한 컬럼 제거
def get_preprocessed_dataset_4(all_df):
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"] = all_df["alone"].fillna(0)
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

# 전처리 단계 5: 호칭의 종류를 줄이고 결측치 처리
def get_preprocessed_dataset_5(all_df):
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"] = all_df["Embarked"].fillna("missing")

    return all_df

# 전처리 단계 6: 범주형 데이터를 수치형 데이터로 변환
def get_preprocessed_dataset_6(all_df):
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

# PyTorch 신경망 모델 정의
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    # 신경망 계층 정의
    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    # 순전파 수행
    x = self.model(x)
    return x

# 테스트 데이터셋을 사용하여 예측 결과를 출력하는 함수
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2)
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())

# 메인 함수: 데이터셋 로딩 및 모델 학습 준비
if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  # 훈련 데이터셋의 샘플 출력
  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  # 데이터 로더 설정
  train_data_loader


train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 24.0000,  2.0000,  0.0000, 24.1500,  2.0000, 13.3029,
         2.0000,  2.0000,  0.0000]): 0
1 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  8.6625,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([  1.0000,   0.0000,  30.0000,   0.0000,   0.0000, 106.4250,   0.0000,
         87.5090,   1.0000,   0.0000,   1.0000]): 1
3 - tensor([ 3.0000,  0.0000, 39.0000,  1.0000,  5.0000, 31.2750,  2.0000, 13.3029,
         3.0000,  6.0000,  0.0000]): 0
4 - tensor([ 3.0000,  1.0000, 29.0000,  1.0000,  0.0000,  7.0458,  2.0000, 13.3029,
         2.0000,  1.0000,  0.0000]): 0
5 - tensor([ 1.0000,  1.0000, 29.0000,  0.0000,  0.0000,  0.0000,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 2.0000,  1.0000,  8.0000,  1.0000,  1.0000, 36.7500,  2.0000, 21.1792,
         0.0000,  2.0000,  0.0000]): 1
7 - te

In [12]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd
import os
from pathlib import Path

BASE_PATH = str(Path(os.getcwd()).resolve().parent.parent.parent)
print(BASE_PATH, "!!!!!!!")

# Titanic 데이터셋을 로드하고 전처리하는 함수
def get_data():
    # Titanic 데이터 로드 및 전처리 (기존 전처리 코드를 사용)
    CURRENT_FILE_PATH = os.getcwd()
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    all_df = pd.concat([train_df, test_df], sort=False)

    # 전처리 단계 적용 (기존 전처리 함수 사용)
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    
    # 데이터 로더 생성
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
    test_data_loader = DataLoader(dataset=TitanicTestDataset(test_X.values), batch_size=len(test_X))

    return train_data_loader, validation_data_loader, test_data_loader


# 모델 구성 내의 Activation Function 변경 가능하도록 수정
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    activation_fn = {
        'relu': nn.ReLU(),
        'elu': nn.ELU(),
        'leaky_relu': nn.LeakyReLU(),
        'prelu': nn.PReLU(),
    }[wandb.config.activation_function]

    # 모델 정의
    self.model = nn.Sequential(
      nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
      activation_fn,
      nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
      activation_fn,
      nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

# 모델 및 옵티마이저 설정 함수
def get_model_and_optimizer():
  my_model = MyModel(n_input=11, n_output=1)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)
  return my_model, optimizer

# 학습 루프 함수
def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.BCEWithLogitsLoss()  # 이진 분류를 위한 손실 함수
  next_print_epoch = 100
  best_validation_loss = float('inf')
  best_model_state = None

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      input, target = train_batch['input'], train_batch['target'].float().unsqueeze(1)
      output_train = model(input)
      loss = loss_fn(output_train, target)
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        input, target = validation_batch['input'], validation_batch['target'].float().unsqueeze(1)
        output_validation = model(input)
        loss = loss_fn(output_validation, target)
        loss_validation += loss.item()
        num_validations += 1

    # Wandb 로그 기록
    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

    # Validation Loss가 가장 낮은 에포크에서 모델 저장
    avg_validation_loss = loss_validation / num_validations
    if avg_validation_loss < best_validation_loss:
        best_validation_loss = avg_validation_loss
        best_model_state = model.state_dict()

    if epoch >= next_print_epoch:
      print(
        f"Epoch {epoch}, "
        f"Training loss {loss_train / num_trains:.4f}, "
        f"Validation loss {loss_validation / num_validations:.4f}"
      )
      next_print_epoch += 100

  # 최상의 모델 상태로 복원
  model.load_state_dict(best_model_state)
  # 최상의 모델로 테스트 데이터에 대해 예측 수행 및 CSV 파일 생성
  evaluate_and_save(model, test_data_loader)


# 메인 함수
def main(args):
  current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

  config = {
    'epochs': args.epochs,
    'batch_size': args.batch_size,
    'learning_rate': 1e-3,
    'n_hidden_unit_list': [20, 20],
    'activation_function': args.activation_function  # Activation Function 설정
  }

  wandb.init(
    mode="online" if args.wandb else "disabled",
    project="titanic_model_training",
    notes="Titanic dataset training with different activation functions",
    tags=["titanic", "activation_function"],
    name=current_time_str,
    config=config
  )
  print(args)
  print(wandb.config)

  # 데이터셋 로드 (테스트 데이터 포함)
  train_data_loader, validation_data_loader, test_data_loader = get_data()

  linear_model, optimizer = get_model_and_optimizer()

  print("#" * 50, 1)

  training_loop(
    model=linear_model,
    optimizer=optimizer,
    train_data_loader=train_data_loader,
    validation_data_loader=validation_data_loader,
    test_data_loader=test_data_loader  # 추가된 부분
  )
  wandb.finish()


# 실행 부분
if __name__ == "__main__":
    # argparse를 사용하지 않고 인수를 직접 설정
    class Args:
        wandb = True  # 또는 False로 설정
        batch_size = 512
        epochs = 1000
        activation_function = 'relu'  # 선택 가능: 'relu', 'elu', 'leaky_relu', 'prelu'

    args = Args()
    main(args)


/Users/kimseongmin/PycharmProjects !!!!!!!


{'epochs': 1000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20], 'activation_function': 'relu'}
################################################## 1
Epoch 100, Training loss 0.6115, Validation loss 0.5837
Epoch 200, Training loss 0.6044, Validation loss 0.5608
Epoch 300, Training loss 0.5936, Validation loss 0.5553
Epoch 400, Training loss 0.6065, Validation loss 0.5514
Epoch 500, Training loss 0.5921, Validation loss 0.5491
Epoch 600, Training loss 0.6092, Validation loss 0.5460
Epoch 700, Training loss 0.5866, Validation loss 0.5450
Epoch 800, Training loss 0.5959, Validation loss 0.5446
Epoch 900, Training loss 0.5952, Validation loss 0.5423
Epoch 1000, Training loss 0.5887, Validation loss 0.5414
Submission file saved as submission.csv


Epoch,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█
Training loss,█▆▇▅▆▄▄▄▃▄▃▃▃▄▃▃▂▃▂▂▂▃▃▂▃▂▁▁▁▂▂▂▃▂▂▂▂▃▂▁
Validation loss,█▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,1000
Training loss,0.58873
Validation loss,0.54141


![My Kaggle Score](https://drive.google.com/thumbnail?id=1H6wuNYiQb5okaomavl4p3Pn9XXWOcwFM)

이번 과제는 머신러닝의 기본적인 훈련부터 모델 최적화, 결과 평가에 이르기까지의 모든 단계를 다룰 수 있는 귀중한 경험이었으며, 
실제 데이터를 다루면서 학습한 이론을 실전에 적용할 수 있는 기회가 되었습니다. 
앞으로 더 복잡한 데이터와 모델을 다룰 때에도 이번 과제에서 배운 경험이 큰 도움이 될 것이라고 생각합니다.